# Filtering JetStream Events

In this notebook, we'll filter events from the Redis Stream that we created in the previous notebook. We'll use a combination of techniques to filter the events:

1. Deduplication using Redis Bloom Filter to avoid processing the same event multiple times
2. Content-based filtering using a machine learning model to identify software-related posts
3. Storing filtered events in Redis for further processing

Redis Bloom Filter is a probabilistic data structure that allows us to check if an element is in a set. It's very memory efficient and has a constant time complexity for both insertion and lookup operations. The trade-off is that it can have false positives, but the probability of false positives can be controlled by the size of the filter.

Machine learning models can be used to classify text into different categories. In this notebook, we'll use a pre-trained zero-shot classification model to classify posts as software-related or not.

## Consuming from Redis Streams

### Model Redis Streams Event
In this section, we'll define a data class to represent the events stored in the Redis Stream. This model will be used to deserialize the events from the stream.

In [1]:
@file:DependsOn("redis.clients:jedis:6.0.0")

In [2]:
import redis.clients.jedis.resps.StreamEntry

data class Event(
    val did: String,
    val rkey: String,
    val text: String,
    val timeUs: String,
    val operation: String,
    val uri: String,
    val parentUri: String,
    val rootUri: String,
    val langs: List<String>,
) {
    companion object {
        fun fromMap(entry: StreamEntry): Event {
            val fields = entry.fields
            return Event(
                did = fields["did"] ?: "",
                rkey = fields["rkey"] ?: "",
                text = fields["text"] ?: "",
                timeUs = fields["timeUs"] ?: "",
                operation = fields["operation"] ?: "",
                uri = fields["uri"] ?: "",
                parentUri = fields["parentUri"] ?: "",
                rootUri = fields["rootUri"] ?: "",
                langs = fields["langs"]?.replace("[", "")?.replace("]", "")?.split(", ") ?: emptyList()
            )
        }
    }
}

### Creating a Redis Client
Create a Jedis client to connect to Redis. This is a reusable client that can be used to interact with Redis Streams.

In [3]:
import redis.clients.jedis.JedisPooled

val jedisPooled = JedisPooled()

### Creating a Consumer Group
Create a consumer group to read from the Redis Stream. A consumer group allows multiple consumers to read from the same stream without duplicating the work. Each consumer in the group will receive a different subset of the messages.

A consumer group can be created in Redis with the XGROUP CREATE command:

`XGROUP CREATE streamName groupName id [MKSTREAM]`

To create a consumer group in this notebook, we will encapsulate the command in a function. The function will take the stream name and the group name as parameters.

In [4]:
import redis.clients.jedis.StreamEntryID

fun createConsumerGroup(streamName: String, consumerGroupName: String) {
    try {
        jedisPooled.xgroupCreate(streamName, consumerGroupName, StreamEntryID("0-0"), true)
    } catch (_: Exception) {
        println("Group already exists")
    }
}

In [5]:
createConsumerGroup("jetstream", "printer-example")

Group already exists


### Reading from the Stream
Create a reusable function to read from the stream. This function will read from the stream and return a list of entries. It uses the XREADGROUP command to read from the stream as part of a consumer group:

`XREADGROUP GROUP groupName consumerName COUNT count BLOCK blockTime streamName id`

The command will be encapsulated in a function that takes the stream name, consumer group name, consumer name, and count as parameters. The function will return a list of entries.

In [6]:
import redis.clients.jedis.params.XReadGroupParams

fun readFromStream(
    streamName: String,
    consumerGroup: String,
    consumer: String, count: Int
): List<Map.Entry<String, List<StreamEntry>>> {
    return jedisPooled.xreadGroup(
        consumerGroup,
        consumer,
        XReadGroupParams().count(count),
        mapOf(
            streamName to StreamEntryID.XREADGROUP_UNDELIVERED_ENTRY
        )
    ) ?: emptyList()
}

### Acknowledging Messages
Create a function to acknowledge the message. This is important to let Redis know that the message has been processed successfully, so it won't be delivered to other consumers in the group.

This is done by using the XACK command:

`XACK streamName groupName id`

The command will be encapsulated in a lambda function that takes the stream name, consumer group name, and entry as parameters. The function will acknowledge the message by calling the XACK command.

In [7]:
fun ackFunction(): (String, String, StreamEntry) -> Unit {
    return { streamName, consumerGroup, entry ->
        jedisPooled.xack(
            streamName,
            consumerGroup,
            entry.id
        )
    }
}

### Consuming the Stream
Create a reusable function to consume the stream.

This function implements a pipeline pattern where each event is processed sequentially by a series of handlers. If any handler returns false, the processing stops for that event.

After processing the event, the function acknowledges the message using the ack function.

In [8]:
%use coroutines

In [9]:
import kotlinx.coroutines.*

fun consumeStream(
    streamName: String,
    consumerGroup: String,
    consumer: String,
    handlers: List<(Event) -> Pair<Boolean, String>>,
    ackFunction: ((String, String, StreamEntry) -> Unit),
    count: Int = 5,
    limit: Int = 5
) {
    var lastMessageTime = System.currentTimeMillis()
    var consumed = 0

    while (consumed < limit) {
        val entries = readFromStream(streamName, consumerGroup, consumer, count)
        val allEntries = entries.flatMap { it.value }
        allEntries.map { entry ->
            consumed++
            val event = Event.fromMap(entry)

            for (handler in handlers) {
                val (shouldContinue, message) = handler(event)
                ackFunction(streamName, consumerGroup, entry)

                if (!shouldContinue) {
                    println("$consumer: Handler stopped processing: $message")
                    break
                }
            }
        }

        if (allEntries.isEmpty()) {
            val now = System.currentTimeMillis()
            if (now - lastMessageTime >= 2_000) {
                println("$consumer: No new messages for 2 seconds. Stopping.")
                break
            }
        }
    }

}

To test the consumeStream function, we'll create a simple handler that prints the event's URI.

In [10]:
val printUri: (Event) -> Pair<Boolean, String> = {
    println("Got event from ${it.uri}")
    Pair(true, "OK")
}

In [11]:
runBlocking {
    consumeStream(
        streamName = "jetstream",
        consumerGroup = "printer-example",
        consumer ="printer-1",
        handlers = listOf(printUri),
        ackFunction = ackFunction(),
        count = 100,
        limit = 100
    )
}

printer-1: No new messages for 2 seconds. Stopping.


## Deduplication with Bloom Filter
Redis Bloom Filter is a probabilistic data structure that allows us to check if an element is in a set. It's very memory efficient and has a constant time complexity for both insertion and lookup operations.


### Creating a Bloom Filter
This function creates a Bloom Filter with the given name. The filter is configured with an error rate of 0.01 and an initial capacity of 1,000,000 elements.

In [12]:
import redis.clients.jedis.bloom.BFReserveParams
import redis.clients.jedis.exceptions.JedisDataException
fun createBloomFilter(name: String) {
    try {
        val errorRate = 0.01
        val capacity = 1_000_000L
        val reserveParams = BFReserveParams().expansion(2)
        jedisPooled.bfReserve(name, errorRate, capacity, reserveParams)
    } catch (_: JedisDataException) {
        println("Bloom filter already exists")
    }
}

### Deduplication Handler
This function creates a handler that checks if an event has already been processed by checking if its URI is in the Bloom Filter. If the URI is in the filter, the handler returns false, which stops the processing of the event.


In [13]:
fun deduplicate(bloomFilter: String): (Event) -> Pair<Boolean, String> {
    return { event ->
        if (jedisPooled.bfExists(bloomFilter, event.uri)) {
            Pair(false, "${event.uri} already processed")
        } else {
            Pair(true, "OK")
        }
    }
}

### Atomic Acknowledgment and Bloom Filter Update
This function creates a handler that acknowledges the message and adds the URI to the Bloom Filter in a single atomic transaction. This ensures that if the acknowledgment succeeds, the URI is also added to the filter, and vice versa.


In [14]:
import redis.clients.jedis.Connection
import redis.clients.jedis.JedisPool
import redis.clients.jedis.Transaction

val jedisPool = JedisPool()

fun ackAndBfFn(bloomFilter: String):  (String, String, StreamEntry) -> Unit {
    return { streamName, consumerGroup, entry ->
        jedisPool.resource.use { jedis ->
            // Create a transaction
            val multi = jedis.multi()

            // Acknowledge the message
            multi.xack(
                streamName,
                consumerGroup,
                entry.id
            )

            // Add the URI to the bloom filter
            multi.bfAdd(bloomFilter, Event.fromMap(entry).uri)

            // Execute the transaction
            multi.exec()
        }
    }
}

In [15]:
createConsumerGroup("jetstream", "deduplicate-example")

Group already exists


In [16]:
val bloomFilterName = "processed-uris"
createBloomFilter("processed-uris")

Bloom filter already exists


In [17]:
runBlocking {
    consumeStream(
        streamName = "jetstream",
        consumerGroup = "deduplicate-example",
        consumer = "deduplicate-1",
        handlers = listOf(deduplicate(bloomFilterName), printUri),
        ackFunction = ackAndBfFn(bloomFilterName),
        count = 100,
        limit = 100
    )
}

deduplicate-1: No new messages for 2 seconds. Stopping.


## Content-Based Filtering with Machine Learning
In this section, we'll use a machine learning model to filter posts based on their content. We'll use a pre-trained zero-shot classification model to classify posts as software-related or not.

### Setting Up the Machine Learning Model
To load the model, we'll use the DJL (Deep Java Library) library. DJL is a high-level framework for deep learning in Java that provides a simple and consistent API for loading and using models.

In [18]:
@file:DependsOn("ai.djl.huggingface:tokenizers:0.33.0")
@file:DependsOn("ai.djl.pytorch:pytorch-engine:0.33.0")
@file:DependsOn("ai.djl:api:0.33.0")
@file:DependsOn("ai.djl:model-zoo:0.33.0")

### Creating the Model Criteria
The criteria is used to load the model and create a predictor. The criteria specifies the model path, the engine to use (in this case, PyTorch), and the translator to use.


In [20]:
import ai.djl.huggingface.tokenizers.HuggingFaceTokenizer
import ai.djl.huggingface.translator.ZeroShotClassificationTranslator
import ai.djl.huggingface.translator.ZeroShotClassificationTranslatorFactory
import ai.djl.modality.nlp.translator.ZeroShotClassificationInput
import ai.djl.modality.nlp.translator.ZeroShotClassificationOutput
import ai.djl.repository.zoo.Criteria
import dev.raphaeldelio.CustomZeroShotClassificationTranslator
import java.nio.file.Paths
import kotlin.io.path.listDirectoryEntries

val tokenizer = HuggingFaceTokenizer.newInstance(Paths.get("/Users/raphaeldelio/Documents/GitHub/redis/kotlinconf-bsky-bot/kotlin-notebooks/notebooks/resources/model/DeBERTa-v3-large-mnli-fever-anli-ling-wanli/tokenizer.json"))

val translator = CustomZeroShotClassificationTranslator.builder(tokenizer).build()

val criteria: Criteria<ZeroShotClassificationInput, ZeroShotClassificationOutput> = Criteria.builder()
    .setTypes(
        ZeroShotClassificationInput::class.java,
        ZeroShotClassificationOutput::class.java
    )
    .optModelPath(Paths.get("resources/model/DeBERTa-v3-large-mnli-fever-anli-ling-wanli"))
    .optEngine("PyTorch")
    .optTranslator(translator)
    .build()


### Loading the Model
Now we'll load the model and create a predictor. The predictor is used to make predictions with the model.


In [21]:
import ai.djl.repository.zoo.ModelZoo

val model = ModelZoo.loadModel(criteria)
val predictor = model.newPredictor()

### Creating a Classification Function
Now we'll create a function to classify text using the model.

The function takes a text as input and returns a classification output. The classification output contains the probabilities for each candidate label.


In [22]:
import ai.djl.modality.nlp.translator.ZeroShotClassificationOutput

fun classify(premise: String): ZeroShotClassificationOutput {
    val candidateLabels = listOf("politics", "american politics", "european politics")
    val input = ZeroShotClassificationInput(premise, candidateLabels.toTypedArray(), true, "{}")
    return predictor.predict(input)
}

### Creating a Filter Handler
Now we'll create a handler that filters events based on their content.

The handler uses the classification function to determine if a post is software-related.

If the post is not software-related, the handler returns false, which stops the processing of the event.


In [23]:
val filter: (Event) -> Pair<Boolean, String> = { event ->
    if (event.text.isNotBlank() && event.operation != "delete") {
        val classification = classify(event.text)
        if (classification.scores.any { it > 0.90 }) {
            Pair(true, "OK")
        } else {
            Pair(false, "Not a post related to politics")
        }
    } else {
        Pair(false, "Text is null or empty")
    }
}

## Storing Filtered Events
In this section, we'll store the filtered events in Redis for further processing.


### Converting Events to Maps
First, we need a function to convert an Event object to a Map that can be stored in Redis as a Hash.

In [25]:
import dev.raphaeldelio.*

fun Event.toMap() = mapOf(
    "did" to this.did,
    "timeUs" to this.timeUs,
    "text" to this.text,
    "langs" to this.langs.joinToString("|"),
    "operation" to this.operation,
    "rkey" to this.rkey,
    "parentUri" to this.parentUri,
    "rootUri" to this.rootUri,
    "uri" to this.uri
)

### Storing Events in Redis
Now we'll create a handler that stores events in Redis. The handler stores the event as a hash in Redis, with the key being the event's URI.


In [26]:
val storeEvent: (Event) -> Pair<Boolean, String> = { event ->
    jedisPooled.hset("post:" + event.uri, event.toMap())
    Pair(true, "OK")
}

### Adding Filtered Events to a New Stream
Finally, we'll create a handler that adds filtered events to a new stream. This allows other consumers to process only the filtered events, rather than having to filter the events themselves.


In [27]:
import redis.clients.jedis.params.XAddParams

val addFilteredEventToStream: (Event) -> Pair<Boolean, String> = { event ->
    jedisPooled.xadd(
        "filtered-events",
        XAddParams.xAddParams()
            .id(StreamEntryID.NEW_ENTRY)
            .maxLen(1_000_000)
            .exactTrimming(),
        event.toMap()
    )
    Pair(true, "OK")
}

In [28]:
createConsumerGroup("jetstream", "store-example")

Group already exists


In [29]:
val bloomFilterName = "store-bf"
createBloomFilter(bloomFilterName)

Bloom filter already exists


## Putting It All Together
Now we'll put all the pieces together to create a complete pipeline for filtering events from the Redis Stream.

In this example we create two consumers that will process the same stream.
- By doing that, we can scale the processing of the events by adding more consumers to the group.
- Redis will make sure that each consumer will receive different messages.


In [30]:
runBlocking {
        listOf(
            async(Dispatchers.IO) {
                consumeStream(
                    streamName = "jetstream",
                    consumerGroup = "store-example",
                    consumer = "store-1",
                    handlers = listOf(
                        deduplicate(bloomFilterName),
                        filter,
                        printUri,
                        storeEvent,
                        addFilteredEventToStream
                    ),
                    ackFunction = ackAndBfFn(bloomFilterName),
                    count = 5,
                    limit = 50
                )
            },
            async(Dispatchers.IO) {
                consumeStream(
                    streamName = "jetstream",
                    consumerGroup = "store-example",
                    consumer = "store-2", // Different consumer
                    handlers = listOf(
                        deduplicate(bloomFilterName),
                        filter,
                        printUri,
                        storeEvent,
                        addFilteredEventToStream
                    ),
                    ackFunction = ackAndBfFn(bloomFilterName),
                    count = 5,
                    limit = 50
                )
            }
        ).awaitAll()
}

store-1: No new messages for 2 seconds. Stopping.
store-2: No new messages for 2 seconds. Stopping.


[kotlin.Unit, kotlin.Unit]

## Next Steps
In the next notebook, we'll enrich the filtered events with additional information, such as topic modeling and embeddings for semantic search.
